# Gym environment for a High Contrast Imaging instrument's AO control system.

## Attributes:
This is a simplified HCI system environment. The dynamics of the environment is based on a Markov Decision Process. The relevant attributes are the "state", "action", "reward".  
The state in this case consists of 2 channels. The first is the WFS recorded phase projected on the DM actuator mode basis. For example, with a 25x25 actuator DM the state is a 25x25 matrix of actuator values. The second is the current DM actuator amplitudes. So overall, the state is a 25x25x2 tensor.   
The action is the actuator amplitudes to be applied on the DM for the current timestep. In the case of a DM with 25x25 actuators, it is simply a 25x25 matrix.    
The reward is a single scalar value. We can use the Strehl, the contrast, or any other such metric that needs to be maximized through AO control.  

## Description:
1. The optical system is defined in a class that inherits from OpenAI's gym environment class.
2. This class contains 3 main methods - \_\_init\_\_(...), step(...), reset(...).
3. The \_\_init\_\_(...) method sets up the optical system with the following components in order: a turbulence generator, a demagnifier, a DM, a WFS, a coronagraph, a quasi-static NCPA aberration, and a detector.
4. The step(action) function does the following:
   * The DM is updated with the actuator values specified in the "action".
   * The wavefront is propagated through the science optical path.
   * The instantaneous focal plane image is calculated.
   * A reward is calculated using the metric function on the focal plane image, and is used to update the reward variable.
   * The timestep is incremented by 1, and the turbulence generator is evolved accordingly.
   * The WFS measurement is read out. The state variable for WFS measurement is updated.
5. A reinforcement learning algorithm works in this environment to maximize expected future reward.

### Step 0: ONLY RUN THIS ON COLAB!

In [ ]:
!pip uninstall hcipy
!rm -rf hcipy
!git clone https://github.com/ehpor/hcipy.git
!cd hcipy; git pull
!cd hcipy; python setup.py install

!mkdir agents
!mkdir models
!mkdir envs
!mkdir utils
!mkdir coronagraphs

!mv agent.py agents/.
!mv model.py models/.
!mv HCI_TestBench.py envs/.
!mv noise_model.py utils/.
!mv replay_buffer.py utils/.
!mv *.fits coronagraphs/.

Now restart the kernel before running the next cell.

### Step 1: Importing necessary libraries.

In [ ]:
# Necessary imports
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
from hcipy import *
import matplotlib.pyplot as plt
from astropy.io import fits
import os, glob
import time
import tensorflow as tf

In [ ]:
from envs.HCI_TestBench import HCI_TestBench

app_amp_file = "coronagraphs/Square_20_80_20_25_0_2_amp_resampled_512.fits"
app_phase_file = "coronagraphs/Square_20_80_20_25_0_2_phase_resampled_512.fits"

### Step 2: Define testbench parameters.

In [ ]:
# Create aperture and pupil/focal grids
wavelength = 532e-9
N = 256
D = 10.5e-3
pupil_grid = make_pupil_grid(N, D*1.1)
science_focal_grid = make_focal_grid(8, 20, wavelength/D)
aperture = circular_aperture(D)

# Telescope parameters
Dtel = 1
tel_pupil_grid = make_pupil_grid(N, Dtel)
tel_aperture = circular_aperture(Dtel)

# Create the deformable mirror
num_actuators = 25
xinetics_basis = make_xinetics_influence_functions(pupil_grid, num_actuators, D * 1.1 / num_actuators)
dm = DeformableMirror(xinetics_basis)
num_modes = len(dm.influence_functions)
dm.actuators = np.zeros(num_modes)

## Create the Shack-Hartmann wavefront sensor and estimator
## Create the microlens array
F_mla = 30. / 0.3
N_mla = 32
D_mla = 10.5e-3

shwfs = SquareShackHartmannWavefrontSensorOptics(pupil_grid, F_mla, N_mla, D_mla)
shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index, circular_aperture(D)(shwfs.mla_grid).astype('bool'))

# Atmosphere parameters
velocity = 10 #m/s
L0 = 40 # outer scale
r0 = 0.2 # Fried parameter
height = 0 # layer height
timestep = 1e-3 # 1 ms per phasescreen

# Make atmosphere
layers = []
layer = InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity, height, stencil_length=2, use_interpolation=True)
layers.append(layer)
atmosphere = MultiLayerAtmosphere(layers, False)

## Create a demagnifier
demag = Magnifier(D / Dtel)

# Make initial phasescreen
wf_tel = Wavefront(tel_aperture(tel_pupil_grid), wavelength)
wf_tel.total_power = 100000
wf = demag.forward(wf_tel)

## Create propagator from pupil to focal plane
prop = FraunhoferPropagator(pupil_grid, science_focal_grid)

## Get the app coronagraph
app_amp = fits.getdata(app_amp_file).ravel()
app_phase = fits.getdata(app_phase_file).ravel()
app = Apodizer(app_amp * np.exp(1j * app_phase))

# For now we don't use a coronagraph so make it just a flat phase
app = Apodizer(np.exp(1j * np.zeros(wf.phase.shape)))

## Create detector
science_camera = NoiselessDetector()

## Generate a diffraction limited image for metrics
diff_lim_img = prop(wf).power

## Get the unit lambda/D
l_D = wavelength / D
plot_grid = make_focal_grid(8, 20, 1)

## Create a noiseless camera image from the perfectly flat wavefront with coronograph
wfdm = dm.forward(wf)
wfapp = app.forward(wfdm)
imapp = prop(wfapp).power
dz_ind = np.where((imapp.grid.x >= (2 * l_D)) &\
                  (imapp.grid.x <= (8 * l_D)) &\
                  (imapp.grid.y >= (-3 * l_D)) &\
                  (imapp.grid.y <= (3 * l_D)))

## Create an NCP aberration
num_coeffs = 9
plaw_index = -1
np.random.seed(7)
coeffs = ((np.random.rand(num_coeffs) - 0.5) * 2) * (np.arange(num_coeffs, dtype=float) + 1) ** plaw_index
coeffs = np.zeros(coeffs.shape) # Set NCP to zero for now
zernike_basis = make_zernike_basis(num_coeffs, D, pupil_grid, 2)
ncp_phase = np.dot(zernike_basis.transformation_matrix, coeffs)
ncp = Apodizer(np.exp(1j * ncp_phase))

# Create an estimate of the NCP aberration for the forward model
ncp_field_est = np.exp(1j * np.zeros(app_phase.shape))
estimated_coeffs = np.zeros(coeffs.shape)

In [ ]:
tb = HCI_TestBench(wf_tel, timestep, atmosphere, demag, dm, shwfs, shwfse, ncp, app, prop, science_camera, dz_ind)

### Step 3: See a standard linear controller in action
In this section we will simply add the wavefront sensor measurement with the current DM actuator values. This should give a reasonably high Strehl / contrast. This is just to see that the testbench works as expected.

In [ ]:
state = tb.reset()

In [ ]:
state = tb.reset() # Start with a new phasescreen

for time in range(500):
    wfs_measurement = state[:, :, 0]
    current_dm_acts = state[:, :, 1]
    
    amps = wfs_measurement + current_dm_acts
    amps = np.squeeze(amps)
    amps = np.clip(amps, -1, 1)
    
    next_state, reward, done, _ = tb.step(amps)
    
    state = next_state.copy()
    
    print("\rTime step: {} Strehl ratio: {:.3f}".format(time, reward), end="")

So with a simple addition operation the resulting Strehl is quite good, between 78 to 80 percent. We should be able to reproduce this with a well trained agent.

### Step 4: Train the DDPG agent

In [ ]:
from agents.agent import DDPG
from collections import deque

In [ ]:
# Define all hyperparameters here
ACTOR_LR = 5e-5
CRITIC_LR = 1e-3
RANDOM_SEED = 42
MU = 0.0
SIGMA = 1.0
BUFFER_SIZE = 1e6
BATCH_SIZE = 256
GAMMA = 0.99
TAU = 1e-3
N_TIME_STEPS = 1
N_LEARN_UPDATES = 1
EPSILON_START = 1.0
EPSILON_DECAY = 1e-9
EPSILON_END = 0.1

if tf.test.is_gpu_available():
    DEVICE = "/GPU:0"
else:
    DEVICE = "/device:CPU:0"

In [ ]:
state_size = (25, 25, 2)
action_size = (25, 25, 1)

In [ ]:
agent = DDPG(state_size, action_size, ACTOR_LR, CRITIC_LR,
             RANDOM_SEED, MU, SIGMA, BUFFER_SIZE, BATCH_SIZE,
             EPSILON_START, EPSILON_END, EPSILON_DECAY,
             GAMMA, TAU, N_TIME_STEPS, N_LEARN_UPDATES, DEVICE)

In [ ]:
def ddpg(n_episodes=100000, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    
    for i_episode in range(1, n_episodes+1):
        state = tb.reset()
        agent.reset()
        score = 0
        t = 0
        
        while(t < 2000):
            t += 1
            action = agent.act(state)
            next_state, reward, done, _ = tb.step(action)
            agent.step(t, state, action, np.log10(reward), next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score / t)
        scores.append(score / t)
        print('\rEpisode {}\t Time in ms:{}\t Average Score: {:.2f}'.format(i_episode, t, np.mean(scores_deque)), end="")
        agent.actor_local.model.save('checkpoint_actor.h5')
        agent.critic_local.model.save('checkpoint_critic.h5')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
        if np.mean(scores_deque) >= 0.8:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            agent.actor_local.model.save('checkpoint_actor.h5')
            agent.critic_local.model.save('checkpoint_critic.h5')
            break
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()